In [1]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

In [3]:
seed = None     # random seed
n = 100         # dimension
u = 1.          # upperbound
epsilon = 0.1   #
np.random.seed(seed)

In [4]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1

loading from: 
/Users/gerardozinno/repos/ProgettoML/cm/data/ML-CUP19-TR.csv


In [5]:
epsilon = 0.1
solvers.options['show_progress'] = False
def solve_with_cvxopt(K, target, size, u):
    A = np.block([[K, -K], [-K, K]])
    b = np.block([epsilon - target, epsilon + target])
    a = np.block([np.ones(n), -np.ones(n)])
    _G, _A, _h, _b = build_problem(2*size, u)
    Q1 = matrix(A)
    q1 = matrix(b)
    G1 = matrix(_G)
    h1 = matrix(_h)
    A1 = matrix(_A)
    b1 = matrix(_b) 
    sol = solvers.qp(Q1, q1, G1, h1, A1, b1)
    return sol['primal objective']

In [6]:
problem_sizes = (100, 200, 300, 400, 500, 600, 700, 800, 900, 1000)
n_prob = 2
problems = [[sample_transform_problem(feat, targ, size, seed) for size in problem_sizes] for _ in range(n_prob)]

In [7]:
ROSEN = 2
GOLDSTEIN = 1

In [8]:
def generate_and_save(feat, targ, size, n=10):
    os.makedirs(f"CSV/{size}/GOLD", exist_ok=True)
    os.makedirs(f"CSV/{size}/ROS", exist_ok=True)
    with open(f"CSV/{size}/optimal_values.txt", "w") as file:
        for i in range(n):
            print(f"generating problem {i}")
            K, target = sample_transform_problem(feat, targ, size)
            print("problem generated\nsoving it with the solver")
            sol = solve_with_cvxopt(K, target, size, u)
            file.write(str(sol)+'\n')
            ds = DataStorer()
            print("About to run GOLDSTEIN")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"CSV/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print("About to run ROSEN")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"CSV/{size}/ROS/{i}.csv")
        

In [9]:
for size in problem_sizes:
    generate_and_save(feat, targ, size, n=5)

generating problem 0
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 1
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 2
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 3
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 4
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 0
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 1
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 2
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN
generating problem 3
problem generated
soving it with the solver
About to run GOLDSTEIN
About to run ROSEN


KeyboardInterrupt: 

In [ ]:
%%time
ds = DataStorer()
prj_type = GOLDSTEIN
sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, verbose=False, prj_type=prj_type, ds=ds)


In [ ]:
ds.save_to_file("GOLDSTEIN_WHOLE_ML_DATASET.csv")
print(f"x: {xg} ({sg})\tv: {vg}\titer: {itg}")

In [ ]:
%%time
ds = DataStorer()
prj_type = ROSEN
sr, xr, vr, itr = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, verbose=False, prj_type=prj_type, ds=ds)
ds.save_to_file("ROSEN.csv")
print(f"x: {xr} ({sr})\tv: {vr}\titer: {itr}")

In [ ]:
np.linalg.norm(vr-vg)

In [ ]:
prj_type = GOLDSTEIN
goldstein_results = []
for run in problems:
    goldstein_temp = []
    for (K, target) in run:
        start = time.time()
        results = (*SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, verbose=False, prj_type=prj_type), time.time()-start)
        goldstein_temp.append(results)
    goldstein_results.append(goldstein_temp) 

In [ ]:
len(goldstein_results)

In [ ]:
zipped_results = [list(zip(*goldstein_results[i])) for i in range(len(goldstein_results))]
iterations = []
times = []
for z in zipped_results:
    iterations.append(z[3])
    times.append(z[4])
gold_average_iter = [sum(x)/len(iterations) for x in zip(*iterations)]
gold_average_time = [sum(x)/len(times) for x in zip(*times)]
print(gold_average_iter, gold_average_time, sep='\n')

In [ ]:
def comparison_plot(element1, label1, element2, label2, xaxis=problem_sizes):
    fix, axs = plt.subplots(ncols=2)
    axs[0].plot(xaxis, element1, '--o')
    axs[0].set_xlabel('problem size')
    axs[0].set_ylabel(label1)
    axs[1].plot(problem_sizes, element2, '--o')
    axs[1].set_xlabel('problem size')
    axs[1].set_ylabel(label2)
    plt.show()


In [ ]:
comparison_plot(gold_average_iter, "number of iterations", gold_average_time, "time (seconds)")

# TODO  
* Calcolare algoritmo su problemi diversi e plottare media
* Vel. conv: prendiamo ottimo risolutore, ad ogni passo plottare valore fun e f* - fi / fi-1 - f*
* Sperimentare con altri parametri
* Confrontare tempi senza numba